In [1]:
# from tensorflow import keras
# dataset = keras.utils.image_dataset_from_directory(
#     "celeba_gan",
#     label_mode=None,
#     # ←----只返回图像，不含标签
#     image_size=(64, 64),
#     batch_size=32,
#     smart_resize=True) 
    # ←----通过裁剪和调整尺寸的巧妙组合，将图像尺寸调整为64×64，同时保持长宽比。我们不希望面部比例被扭曲

In [3]:
import keras
from keras import layers

discriminator = keras.Sequential(
    [
        keras.Input(shape=(64, 64, 3)),
        layers.Conv2D(64, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Conv2D(128, kernel_size=4, strides=2, padding="same"),
        layers.LeakyReLU(alpha=0.2),
        layers.Flatten(),
        layers.Dropout(0.2), 
        # ←----添加dropout层，这是一项重要的技巧
        layers.Dense(1, activation="sigmoid"),
    ],
    name="discriminator",
)

/home/binbin/anaconda3/envs/tf/lib/python3.11/site-packages/keras/src/layers/activations/leaky_relu.py:41: UserWarning: Argument `alpha` is deprecated. Use `negative_slope` instead.
  warnings.warn(
I0000 00:00:1737299266.966523   78739 gpu_device.cc:2022] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 759 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1660 SUPER, pci bus id: 0000:01:00.0, compute capability: 7.5


In [4]:
latent_dim = 128 
# ←----潜在空间由128维向量组成

generator = keras.Sequential(
    [
        keras.Input(shape=(latent_dim,)),
        layers.Dense(8 * 8 * 128), 
        # ←----该层输出的元素个数与编码器Flatten层的输出相同
        layers.Reshape((8, 8, 128)), 
        # ←----编码器Flatten层的逆操作
        layers.Conv2DTranspose(128, 
        kernel_size=4, strides=2, padding="same"),  
        # ←----编码器Conv2D层的逆操作
        layers.LeakyReLU(alpha=0.2), 
        # ←----使用LeakyReLU作为激活函数
        layers.Conv2DTranspose(256,
        kernel_size=4, strides=2, padding="same"),
        # ←----编码器Conv2D层的逆操作
        layers.LeakyReLU(alpha=0.2),
        # ←----使用LeakyReLU作为激活函数
        layers.Conv2DTranspose(512,
        kernel_size=4, strides=2, padding="same"),
        # ←----编码器Conv2D层的逆操作
        layers.LeakyReLU(alpha=0.2),
        # ←----使用LeakyReLU作为激活函数
        layers.Conv2D(3, kernel_size=5, padding="same", activation="sigmoid"), 
        # ←----最终输出形状为(64, 64, 3)
    ],
    name="generator",
)

In [ ]:
import tensorflow as tf
class GAN(keras.Model):
    def __init__(self, discriminator, generator, latent_dim):
        super().__init__()
        self.discriminator = discriminator
        self.generator = generator
        self.latent_dim = latent_dim
        self.d_loss_metric = keras.metrics.Mean(name="d_loss") 
        # ←---- (本行及以下2行)创建指标，跟踪每轮训练的两个损失值
        self.g_loss_metric = keras.metrics.Mean(name="g_loss")

    def compile(self, d_optimizer, g_optimizer, loss_fn):
        super(GAN, self).compile()
        self.d_optimizer = d_optimizer
        self.g_optimizer = g_optimizer
        self.loss_fn = loss_fn

    @property
    def metrics(self):  
        # ←----创建指标，跟踪每轮训练的两个损失值
        return [self.d_loss_metric, self.g_loss_metric]

    def train_step(self, real_images):
        batch_size = tf.shape(real_images)[0]  
        # ←---- (本行及以下2行)在潜在空间中采样随机点
        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim))
        generated_images = self.generator(random_latent_vectors)  
        # ←----将这些点解码为虚假图像
        combined_images = tf.concat([generated_images, real_images], axis=0)  
        # ←----将这些虚假图像与真实图像混合
        labels = tf.concat(  
            # ←---- (本行及以下2行)指定标签，以区分真假图像
            [tf.ones((batch_size, 1)), tf.zeros((batch_size, 1))],
            axis=0
        )
        labels += 0.05 * tf.random.uniform(tf.shape(labels))  
        # ←----向标签中添加随机噪声，这是一项很重要的技巧

        with tf.GradientTape() as tape:  
            # ←---- (本行及以下5行)训练判别器
            predictions = self.discriminator(combined_images)
            d_loss = self.loss_fn(labels, predictions)
        grads = tape.gradient(d_loss, self.discriminator.trainable_weights)
        self.d_optimizer.apply_gradients(
            zip(grads, self.discriminator.trainable_weights)
        )

        random_latent_vectors = tf.random.normal(
            shape=(batch_size, self.latent_dim)) 
        # ←----在潜在空间中采样随机点

        misleading_labels = tf.zeros((batch_size, 1))  
        # ←----指定标签，全部是“真实图像”（这是在撒谎）

        with tf.GradientTape() as tape: 
            # ←---- (本行及以下5行)训练生成器
            predictions = self.discriminator(
                self.generator(random_latent_vectors))
            g_loss = self.loss_fn(misleading_labels, predictions)
        grads = tape.gradient(g_loss, self.generator.trainable_weights)
        self.g_optimizer.apply_gradients(
            zip(grads, self.generator.trainable_weights))

        self.d_loss_metric.update_state(d_loss)
        self.g_loss_metric.update_state(g_loss)
        return {"d_loss": self.d_loss_metric.result(),
                "g_loss": self.g_loss_metric.result()}

In [ ]:
class GANMonitor(keras.callbacks.Callback):
    def __init__(self, num_img=3, latent_dim=128):
        self.num_img = num_img
        self.latent_dim = latent_dim

    def on_epoch_end(self, epoch, logs=None):
        random_latent_vectors = tf.random.normal(
            shape=(self.num_img, self.latent_dim))
        generated_images = self.model.generator(random_latent_vectors)
        generated_images *= 255
        generated_images.numpy()
        for i in range(self.num_img):
            img = keras.utils.array_to_img(generated_images[i])
            img.save(f"generated_img_{epoch:03d}_{i}.png")

In [ ]:
epochs = 100 
# ←---- 20轮过后，就开始得到有趣的结果

gan = GAN(discriminator=discriminator, generator=generator,
          latent_dim=latent_dim)
gan.compile(
    d_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    g_optimizer=keras.optimizers.Adam(learning_rate=0.0001),
    loss_fn=keras.losses.BinaryCrossentropy(),
)

gan.fit(
    dataset, epochs=epochs,
    callbacks=[GANMonitor(num_img=10, latent_dim=latent_dim)]
)